In [25]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [26]:
data = pd.read_csv('DummiesV2.csv')
data.head()

,À l'avant du véhicule,À l'arrière du véhicule,Conducteur,Passager,Indemne,Tué,Blessé hospitalisé,Blessé léger,Homme,Aucun équippement,...,3 voies de circulation,4 voies de circulation,5 voies de circulation,6 voies de circulation,7 voies de circulation,8 voies de circulation,9 voies de circulation,10 voies de circulation,11 voies de circulation,12 voies de circulation
0,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [27]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
targets = ['Tué', 'Indemne', 'Blessé léger', 'Blessé hospitalisé']
X = data.drop(targets, axis=1)
X = pd.DataFrame(data=StandardScaler().fit_transform(X), columns=X.columns)
Y = data[targets].values.argmax(axis=1)
X_train, X_test,Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
estimator = LogisticRegression(multi_class='multinomial', max_iter=2_000)
selector = RFE(estimator, n_features_to_select=50, step=1)
selector = selector.fit(X_train, Y_train)
selector.support_

array([False, False,  True,  True,  True,  True,  True,  True, False,
       False, False, False,  True,  True, False,  True,  True,  True,
        True, False,  True, False, False, False,  True, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True, False, False,  True,  True,  True,  True, False, False,
       False, False, False, False, False, False, False, False,  True,
        True, False, False, False, False,  True, False, False, False,
       False, False, False,  True, False, False,  True, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True, False, False,  True, False, False, False,
        True,  True, False,  True,  True, False,  True,  True, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [31]:
selected = [variable for variable, boolean in zip(X.columns.tolist(), selector.support_.tolist()) if boolean]
selected

['Conducteur',
 'Passager',
 'Homme',
 'Aucun équippement',
 'Ceinture',
 'Casque',
 'Deux-roues non-motorisé',
 'Deux-roues motorisé',
 'Poids lourd',
 'Transport en commun',
 'Véhicule léger',
 "Pas d'obstacle fixe",
 'Obstacle : Arbre',
 'Obstacle : Bâtiment, mur, pile de pont',
 "Pas d'obstacle mobile",
 'Obstacle : Piéton',
 'Obstacle : Véhicule en mouvement',
 'Pas de choc',
 "Choc à l'avant",
 "Choc à l'arrière",
 'Choc latéral',
 'Déporté',
 'En tournant',
 'En ouvrant une porte',
 'Plein jour',
 'Hors agglomération',
 'Place',
 'Collision frontale (2 véhicules)',
 "Collision par l'arrière (2 véhicules)",
 'Collision par le côté (2 véhicules)',
 'Collision en chaîne (3 véhicules ou plus)',
 'Collisions multiples (3 véhicules ou plus)',
 'Autre collision',
 'Autoroute',
 'Voie communale',
 'Circulation bidirectionnelle',
 'Circulation à chaussées séparées',
 'Route plate',
 'Route en pente',
 'Route droite',
 'En virage',
 'Souterrain, tunnel',
 'Vitesse limitée à 30km/h',
 'Vit

In [32]:
np.mean(selector.estimator_.predict(X_test[selected]) == Y_test)

0.6446858809919018

Voir aussi `sklearn.feature_selection.SelectKBest`